# Vectorizers

In this notebook, we will show how to use RedisVL4j to create embeddings using the built-in text embedding vectorizers. Today RedisVL4j supports:
1. OpenAI
2. HuggingFace (Sentence Transformers via ONNX)
3. Cohere
4. VoyageAI
5. LangChain4j (wraps many providers)
6. Custom vectorizers

Before running this notebook, be sure to:
1. Have Java 17+ installed
2. Have a running Redis Stack instance with RediSearch > 2.4 active

For example, you can run Redis Stack locally with Docker:

```bash
docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```

This will run Redis on port 6379 and RedisInsight at http://localhost:8001.

In [ ]:
// Add RedisVL4j dependencies
%maven com.redis:redisvl4j-core:0.1.0-SNAPSHOT

// Import necessary classes
import com.redis.vl.utils.vectorize.*;
import com.redis.vl.index.SearchIndex;
import com.redis.vl.schema.IndexSchema;
import com.redis.vl.query.VectorQuery;
import redis.clients.jedis.JedisPooled;
import java.util.List;
import java.util.Map;
import java.util.Arrays;

## Creating Text Embeddings

This example will show how to create an embedding from 3 simple sentences with a number of different text vectorizers in RedisVL4j.

- "That is a happy dog"
- "That is a happy person"
- "Today is a nice day"

### OpenAI

The `OpenAIVectorizer` makes it simple to use RedisVL4j with the embeddings models at OpenAI. This uses LangChain4j's OpenAI integration under the hood.

You'll need to set your OpenAI API key as an environment variable or pass it in configuration.

In [ ]:
// Setup the API Key
String apiKey = System.getenv("OPENAI_API_KEY");
if (apiKey == null) {
    throw new IllegalStateException("Please set OPENAI_API_KEY environment variable");
}

In [ ]:
// Create a vectorizer using OpenAI's text-embedding-ada-002 model
var oai = OpenAIVectorizer.builder()
    .apiKey(apiKey)
    .modelName("text-embedding-ada-002")
    .build();

// Embed a single sentence
float[] test = oai.embed("This is a test sentence.");
System.out.println("Vector dimensions: " + test.length);

// Print first 10 dimensions
System.out.println(Arrays.toString(Arrays.copyOfRange(test, 0, 10)));

In [ ]:
// Create many embeddings at once
List<String> sentences = Arrays.asList(
    "That is a happy dog",
    "That is a happy person",
    "Today is a sunny day"
);

List<float[]> embeddings = oai.embedBatch(sentences);
System.out.println("Number of embeddings: " + embeddings.size());
System.out.println(Arrays.toString(Arrays.copyOfRange(embeddings.get(0), 0, 10)));

### HuggingFace

[Huggingface](https://huggingface.co/models) is a popular NLP platform that has a number of pre-trained models you can use off the shelf. RedisVL4j supports using Huggingface "Sentence Transformers" via ONNX models to create embeddings from text locally.

Models are automatically downloaded and cached on first use.

In [ ]:
// Create a vectorizer using HuggingFace Sentence Transformers
// Choose your model from the huggingface website
var hf = SentenceTransformersVectorizer.builder()
    .modelName("sentence-transformers/all-mpnet-base-v2")
    .build();

// Embed a sentence
float[] hfTest = hf.embed("This is a test sentence.");
System.out.println("Vector dimensions: " + hfTest.length);
System.out.println(Arrays.toString(Arrays.copyOfRange(hfTest, 0, 10)));

In [ ]:
// You can also create many embeddings at once
List<float[]> hfEmbeddings = hf.embedBatch(sentences);
System.out.println("Created " + hfEmbeddings.size() + " embeddings");

### Cohere

[Cohere](https://dashboard.cohere.ai/) allows you to implement language AI into your product. The `CohereVectorizer` makes it simple to use RedisVL4j with the embeddings models at Cohere via LangChain4j.

You'll need to set your Cohere API key.

In [ ]:
// Setup the API Key
String cohereApiKey = System.getenv("COHERE_API_KEY");
if (cohereApiKey == null) {
    throw new IllegalStateException("Please set COHERE_API_KEY environment variable");
}

Special attention needs to be paid to the input type for each embed call. For example, for embedding queries, you should set input type to `SEARCH_QUERY`; for embedding documents, set to `SEARCH_DOCUMENT`. See more information [here](https://docs.cohere.com/reference/embed)

In [ ]:
import dev.langchain4j.model.cohere.CohereScoringModel.InputType;

// Create a vectorizer
var co = CohereVectorizer.builder()
    .apiKey(cohereApiKey)
    .modelName("embed-english-v3.0")
    .build();

// Embed a search query
float[] queryEmbed = co.embed("This is a test sentence.", InputType.SEARCH_QUERY);
System.out.println("Query vector dimensions: " + queryEmbed.length);
System.out.println(Arrays.toString(Arrays.copyOfRange(queryEmbed, 0, 10)));

// Embed a document
float[] docEmbed = co.embed("This is a test sentence.", InputType.SEARCH_DOCUMENT);
System.out.println("Document vector dimensions: " + docEmbed.length);
System.out.println(Arrays.toString(Arrays.copyOfRange(docEmbed, 0, 10)));

Learn more about using RedisVL and Cohere together through [this dedicated user guide](https://docs.cohere.com/docs/redis-and-cohere).

### VoyageAI

[VoyageAI](https://dash.voyageai.com/) allows you to implement language AI into your product. The `VoyageAIVectorizer` makes it simple to use RedisVL4j with the embeddings models at VoyageAI via LangChain4j.

You'll need to set your VoyageAI API key.

In [ ]:
// Setup the API Key
String voyageApiKey = System.getenv("VOYAGE_API_KEY");
if (voyageApiKey == null) {
    throw new IllegalStateException("Please set VOYAGE_API_KEY environment variable");
}

Special attention needs to be paid to the input type for each embed call. For example, for embedding queries, you should set input type to `QUERY`; for embedding documents, set to `DOCUMENT`. See more information [here](https://docs.voyageai.com/docs/embeddings)

In [ ]:
import dev.langchain4j.model.voyageai.VoyageAiEmbeddingModel.InputType;

// Create a vectorizer
var vo = VoyageAIVectorizer.builder()
    .apiKey(voyageApiKey)
    .modelName("voyage-law-2")  // Check available models at https://docs.voyageai.com/docs/embeddings
    .build();

// Embed a search query
float[] voyageQuery = vo.embed("This is a test sentence.", InputType.QUERY);
System.out.println("Query vector dimensions: " + voyageQuery.length);
System.out.println(Arrays.toString(Arrays.copyOfRange(voyageQuery, 0, 10)));

// Embed a document
float[] voyageDoc = vo.embed("This is a test sentence.", InputType.DOCUMENT);
System.out.println("Document vector dimensions: " + voyageDoc.length);
System.out.println(Arrays.toString(Arrays.copyOfRange(voyageDoc, 0, 10)));

### LangChain4j Vectorizers

RedisVL4j provides a `LangChain4JVectorizer` that wraps any LangChain4j `EmbeddingModel`. This gives you access to many providers including:
- Azure OpenAI
- Google Vertex AI
- Amazon Bedrock
- Mistral AI
- And many more

Example with Azure OpenAI:

In [ ]:
import dev.langchain4j.model.azure.AzureOpenAiEmbeddingModel;

// Create an Azure OpenAI embedding model
var azureModel = AzureOpenAiEmbeddingModel.builder()
    .apiKey(System.getenv("AZURE_OPENAI_API_KEY"))
    .endpoint(System.getenv("AZURE_OPENAI_ENDPOINT"))
    .deploymentName("text-embedding-ada-002")
    .build();

// Wrap it in a LangChain4JVectorizer
var azureVectorizer = new LangChain4JVectorizer(azureModel);

// Use it like any other vectorizer
float[] azureEmbed = azureVectorizer.embed("This is a test sentence.");
System.out.println("Vector dimensions: " + azureEmbed.length);

### Custom Vectorizers

RedisVL4j supports the use of custom vectorizers by implementing the `BaseVectorizer` interface. This enables compatibility with any function that generates vectors from string data.

In [ ]:
// Create a simple custom vectorizer
class CustomVectorizer extends BaseVectorizer {
    public CustomVectorizer() {
        super("custom-model", 768);
    }
    
    @Override
    public float[] embed(String text) {
        float[] embedding = new float[768];
        Arrays.fill(embedding, 0.101f);
        return embedding;
    }
    
    @Override
    public List<float[]> embedBatch(List<String> texts) {
        return texts.stream()
            .map(this::embed)
            .collect(java.util.stream.Collectors.toList());
    }
}

var customVectorizer = new CustomVectorizer();
float[] customEmbed = customVectorizer.embed("This is a test sentence.");
System.out.println(Arrays.toString(Arrays.copyOfRange(customEmbed, 0, 10)));

This enables the use of custom vectorizers with other RedisVL4j components like SemanticCache:

In [ ]:
import com.redis.vl.extensions.cache.SemanticCache;

var jedis = new JedisPooled("localhost", 6379);

var cache = SemanticCache.builder()
    .name("custom_cache")
    .vectorizer(customVectorizer)
    .jedis(jedis)
    .build();

cache.store("this is a test prompt", "this is a test response");
var result = cache.check("this is also a test prompt");
System.out.println("Cache result: " + result);

## Search with Provider Embeddings

Now that we've created our embeddings, we can use them to search for similar sentences. We will use the same 3 sentences from above and search for similar sentences.

First, we need to create the schema for our index.

Here's what the schema for the example looks like in YAML for the HuggingFace vectorizer:

```yaml
version: '0.1.0'

index:
    name: vectorizers
    prefix: doc
    storage_type: hash

fields:
    - name: sentence
      type: text
    - name: embedding
      type: vector
      attrs:
        dims: 768
        algorithm: flat
        distance_metric: cosine
```

In [ ]:
import com.redis.vl.schema.*;
import redis.clients.jedis.search.schemafields.*;

// Create the schema programmatically
var schema = IndexSchema.builder()
    .indexName("vectorizers")
    .prefix("doc")
    .storageType(IndexSchema.StorageType.HASH)
    .addTextField("sentence")
    .addVectorField(VectorField.builder()
        .name("embedding")
        .algorithm(VectorField.Algorithm.FLAT)
        .dimensions(768)
        .distanceMetric(VectorField.DistanceMetric.COSINE)
        .dataType(VectorField.VectorDataType.FLOAT32)
        .build())
    .build();

// Create the index
var index = new SearchIndex(schema, jedis);
index.create(true); // overwrite if exists
System.out.println("Index created: " + index.getName());

Loading data to RedisVL4j is easy. It expects a list of maps. The vector is stored as a float array.

In [ ]:
// Create embeddings for our sentences
List<float[]> sentenceEmbeddings = hf.embedBatch(sentences);

// Prepare data for loading
List<Map<String, Object>> data = new java.util.ArrayList<>();
for (int i = 0; i < sentences.size(); i++) {
    Map<String, Object> doc = new java.util.HashMap<>();
    doc.put("sentence", sentences.get(i));
    doc.put("embedding", sentenceEmbeddings.get(i));
    data.add(doc);
}

// Load data into the index
index.load(data);
System.out.println("Loaded " + data.size() + " documents");

In [ ]:
// Use the HuggingFace vectorizer to create a query embedding
float[] queryEmbedding = hf.embed("That is a happy cat");

// Create and execute a vector query
var query = VectorQuery.builder()
    .vector(queryEmbedding)
    .field("embedding")
    .returnFields(List.of("sentence"))
    .numResults(3)
    .build();

List<Map<String, Object>> results = index.query(query);
for (var doc : results) {
    System.out.println(doc.get("sentence") + " - Distance: " + doc.get("vector_distance"));
}

## Selecting your float data type

When embedding text as vectors, RedisVL4j supports different floating point data types: `FLOAT32`, `FLOAT64`, and `FLOAT16`.

Your data type must match what is defined in your search index. If one is not explicitly set, the default is `FLOAT32`.

In [ ]:
// Create a vectorizer with float16 precision
var vectorizer16 = SentenceTransformersVectorizer.builder()
    .modelName("sentence-transformers/all-mpnet-base-v2")
    .dataType(VectorField.VectorDataType.FLOAT16)
    .build();

float[] float16Embed = vectorizer16.embed("test sentence");

// Create a vectorizer with float64 precision
var vectorizer64 = SentenceTransformersVectorizer.builder()
    .modelName("sentence-transformers/all-mpnet-base-v2")
    .dataType(VectorField.VectorDataType.FLOAT64)
    .build();

float[] float64Embed = vectorizer64.embed("test sentence");

System.out.println("Different data types produce different representations: " + 
    !Arrays.equals(float16Embed, float64Embed));

In [ ]:
// Cleanup
index.delete(true);
System.out.println("Index deleted");